In [1]:
# Import required libraries
import os
import sys
import ipywidgets as widgets
from IPython.display import display, clear_output
import ee
from datetime import datetime, date
from pathlib import Path
import geemap

# Import GeoClimate-Fetcher modules
from geoclimate_fetcher.core import (
    authenticate,
    MetadataCatalog,
    GeometryHandler,
    GEEExporter, 
    ImageCollectionFetcher,
    StaticRasterFetcher
)

from geoclimate_fetcher.ui import (
    AuthWidget, 
    MapWidget,
    DatasetPickerWidget,
    BandPickerWidget, 
    TimeSliderWidget,
    DownloadDialogWidget
)

# Add the state manager - copy this to a file called state_manager.py in the geoclimate_fetcher directory
"""
State manager to ensure geometry is shared between widgets
"""
import ee
from geoclimate_fetcher.core.geometry import GeometryHandler

class GeometryStateManager:
    """Singleton class to store global geometry state across widgets"""
    
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(GeometryStateManager, cls).__new__(cls)
            cls._instance.geometry_handler = GeometryHandler()
            cls._instance.geometry_set = False
            cls._instance.debug_mode = True
        return cls._instance
    
    def set_geometry(self, geo_json=None, geometry=None, name="user_drawn_geometry"):
        """Set the geometry from GeoJSON or direct EE geometry"""
        if self.debug_mode:
            print(f"GeometryStateManager: Setting geometry with name '{name}'")
        
        if geo_json is not None:
            self.geometry_handler.set_geometry_from_geojson(geo_json, name)
            self.geometry_set = True
            
            if self.debug_mode:
                try:
                    area = self.geometry_handler.get_geometry_area()
                    print(f"GeometryStateManager: Set geometry from GeoJSON, area = {area:.2f} km²")
                except Exception as e:
                    print(f"GeometryStateManager: Error calculating area: {str(e)}")
        
        elif geometry is not None:
            self.geometry_handler._current_geometry = geometry
            self.geometry_handler._current_geometry_name = name
            self.geometry_set = True
            
            if self.debug_mode:
                try:
                    area = self.geometry_handler.get_geometry_area()
                    print(f"GeometryStateManager: Set direct geometry, area = {area:.2f} km²")
                except Exception as e:
                    print(f"GeometryStateManager: Error calculating area: {str(e)}")
    
    def get_geometry_handler(self):
        """Get the geometry handler with current geometry"""
        if self.debug_mode:
            print(f"GeometryStateManager: Getting geometry handler, geometry set = {self.geometry_set}")
            if self.geometry_set:
                try:
                    area = self.geometry_handler.get_geometry_area()
                    print(f"GeometryStateManager: Current geometry area = {area:.2f} km²")
                except Exception as e:
                    print(f"GeometryStateManager: Error calculating area: {str(e)}")
        
        return self.geometry_handler
    
    def has_geometry(self):
        """Check if geometry is set"""
        return self.geometry_set

# Create global state manager
state_manager = GeometryStateManager()

# Create core objects
metadata_catalog = MetadataCatalog()
# We'll use the geometry handler from the state manager
exporter = GEEExporter()

# Initialize state variables
auth_complete = False
geometry_complete = False
dataset_selected = False
bands_selected = False
dates_selected = False

# Data parameters
current_dataset = None
selected_bands = []
start_date = None
end_date = None
snippet_type = None
download_path = None

# Custom geometry handler for map_widget
class SharedGeometryHandler(GeometryHandler):
    """A wrapper around GeometryHandler that updates the global state"""
    
    def set_geometry_from_geojson(self, geojson_dict, name="custom_aoi"):
        """Override to update global state"""
        result = super().set_geometry_from_geojson(geojson_dict, name)
        # Update global state
        state_manager.set_geometry(geo_json=geojson_dict, name=name)
        return result
    
    def set_geometry_from_drawn(self, geojson_dict):
        """Override to update global state"""
        result = super().set_geometry_from_drawn(geojson_dict)
        # Update global state
        state_manager.set_geometry(geo_json=geojson_dict, name="drawn_aoi")
        return result

# Callback for geometry selection
def on_geometry_selected(geometry_handler):
    global geometry_complete
    geometry_complete = True
    
    # IMPORTANT: Update the global state with the selected geometry
    if geometry_handler.current_geometry is not None:
        state_manager.set_geometry(
            geometry=geometry_handler.current_geometry, 
            name=geometry_handler.current_geometry_name
        )
    
    print("Geometry selected. Now you can select a dataset.")
    print(f"Global state updated: {state_manager.has_geometry()}")
    dataset_picker.display()

# Initialize map widget with custom geometry handler
local_geometry_handler = SharedGeometryHandler()
map_widget = MapWidget(on_geometry_selected=on_geometry_selected)
# Replace the geometry handler in the map widget with our shared one
map_widget.geometry_handler = local_geometry_handler

# Callback for dataset selection
def on_dataset_selected(dataset):
    global dataset_selected, current_dataset, snippet_type
    dataset_selected = True
    current_dataset = dataset
    snippet_type = dataset.get('Snippet Type')
    # Update band picker and time slider with the selected dataset
    band_picker.set_dataset(dataset)
    time_slider.set_dataset(dataset)
    print(f'Selected dataset: {dataset.get("Dataset Name")} ({snippet_type})')
    print("Please select the bands you want to download.")
    band_picker.display()

# Callback for band selection
def on_bands_selected(bands):
    global bands_selected, selected_bands
    bands_selected = True
    selected_bands = bands
    print(f'Selected bands: {", ".join(bands)}')
    print("Now please select the time range.")
    # Show time slider for ImageCollections, or the download dialog for single Images
    if snippet_type == 'ImageCollection':
        time_slider.display()
    else:
        initialize_download_dialog()

# Initialize dataset picker widget
dataset_picker = DatasetPickerWidget(metadata_catalog, on_dataset_selected=on_dataset_selected)
# Initialize band picker widget
band_picker = BandPickerWidget(metadata_catalog, on_bands_selected=on_bands_selected)

# Callback for date range selection
def on_dates_selected(start_d, end_d):
    global dates_selected, start_date, end_date
    dates_selected = True
    start_date = start_d
    end_date = end_d
    print(f'Selected time range: {start_date} to {end_date}')
    print("Now you can configure the download.")
    initialize_download_dialog()

# Initialize time slider widget
time_slider = TimeSliderWidget(metadata_catalog, on_dates_selected=on_dates_selected)

# Callback for download completion
def on_download_complete():
    print("Download complete!")
    print(f'File saved to: {download_path}')

# Initialize download dialog
def initialize_download_dialog():
    global download_dialog
    # Initialize the download dialog using the state manager's geometry handler
    download_dialog = DownloadDialogWidget(
        metadata_catalog, 
        state_manager.get_geometry_handler(), 
        on_download_complete=on_download_complete
    )
    
    # Set parameters based on previous selections
    dataset_name = current_dataset.get('Dataset Name')
    ee_id = current_dataset.get('Earth Engine ID')
    download_dialog.set_parameters(
        dataset_name, 
        ee_id, 
        selected_bands, 
        start_date, 
        end_date
    )
    
    # Display the download dialog
    download_dialog.display()

# Show the main workflow after authentication
def show_main_workflow():
    if not auth_complete:
        print("Please authenticate first.")
        return
    # Show the AOI selection widget
    clear_output()
    print("Authentication successful. Please select an area of interest.")
    map_widget.display()

# Callback for successful authentication
def on_auth_success(auth):
    global auth_complete
    auth_complete = True
    show_main_workflow()

# Initialize the authentication widget
auth_widget = AuthWidget(on_auth_success=on_auth_success)
auth_widget.display()

Using geemap from standard import
